In [ ]:
#Chargement des packages
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
import lightgbm
import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWg-B2dPb4MUpivKd6_F2Q4USjaKJnDq4XKLZBuaUZSbP2kBomqLWBw
Mounted at /content/drive


In [4]:
# Chargement des données
data_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/X_train_rakuten_afterEDA_preprocessing.csv')
data_train.head()

,designation_orig,nom_img,lang_designation,designation,prdtypecode,designation_token_final
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,image_1263597046_product_3804725264.jpg,de,olivia: notebook personnalise / pages / punks...,10,"['notebook', 'page', 'design']"
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,image_1008141237_product_436067568.jpg,fr,journal des art (le) n° du // art et son ma...,2280,"['journal', 'art', 'art', 'marche', 'salon', '..."
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,image_938777978_product_201115110.jpg,fr,grand stylet ergonomique bleu gamepad nintendo...,50,"['stylet', 'ergonomique', 'gamepad', 'nintendo..."
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,image_457047496_product_50418756.jpg,fr,peluche donald europe disneyland (marionn...,1280,"['peluche', 'donald', 'europe', 'disneyland', ..."
4,La Guerre Des Tuques Luc a des idées de grande...,image_1077757786_product_278535884.jpg,fr,la guerre des tuques luc a des idees de grande...,2705,"['guerre', 'luc', 'idees', 'grandeur', 'vouloi..."


In [5]:
data_train.shape

(84613, 6)

In [6]:
#Definition des features et labels
from tqdm.notebook import tqdm
import re

#Features

data_train['designation_str_tokens'] = ""
for i in tqdm(range(len(data_train))):
    data_train['designation_str_tokens'].iloc[i] += "".join(data_train['designation_token_final'].iloc[i]) 
data_train['designation_str_tokens'] = data_train['designation_str_tokens'].map(lambda x: re.sub(r',', '', x))
data_train['designation_str_tokens'] = data_train['designation_str_tokens'].map(lambda x: re.sub(r"'", '', x))

X_data_train = data_train.designation_str_tokens
X_data_train = X_data_train.str.replace("[", "").str.replace("]", "")

  0%|          | 0/84613 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [7]:
# SANS STEMMATISATION
# Séparation des données
X = data_train.designation
y = data_train.prdtypecode

# Division des données
X_train_brut, X_test_brut, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state = 0)

In [8]:
X_train_brut

19014    decoration de noel femmes regarder la bande de...
72177    transformateur robot piscine dolphin gamme e l...
48245    regles avancees officielles de donjons & drago...
32201    oxford bloc de direction perfore  pages g lign...
40516        lot de  encyclopedie des armes n°  à  et n°  
                               ...                        
21243    hochet anneau adele l&#;elephante   nattou ce ...
45891    chargeur de batterie de voiture port usb siege...
42613                   à l'exterieur declenche: afrique d
43567    poussette baby walker  musicale + poupon minik...
68268                     console ps &#;  jeu &#; manette 
Name: designation, Length: 67690, dtype: object

In [9]:
# AVEC STEMMATISATION
# Séparation des données
X = X_data_train
y = data_train.prdtypecode

# Division des données
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state = 0)

In [10]:
X_train

19014    decoration noel femme regarder bande noel band...
72177    transformateur robot piscine dolphin gamm remp...
48245                   regle officiel donjon dragon maitr
32201    oxford bloc direction perfor page lignees couv...
40516                                lot encyclopedie arme
                               ...                        
21243    hochet anneau adel elephante nattou hochet ann...
45891    chargeur batterie voiture port usb siege charg...
42613                                     exterieur afriqu
43567    poussette baby musical poupon minikis poussett...
68268                                  console jeu manette
Name: designation_str_tokens, Length: 67690, dtype: object

In [11]:
## VECTORISATION DES DONNÉES
# SANS STEMMATISATION
# Application TfidfVectorizer
tfvectorizer = TfidfVectorizer()
# TfidfVectorizer
X_train_tf_brut = tfvectorizer.fit_transform(X_train_brut)
X_test_tf_brut = tfvectorizer.transform(X_test_brut)

X_train_tf_brut.shape

(67690, 114002)

In [12]:
print(X_train_tf_brut)

  (0, 76317)	0.043337151975561664
  (0, 110752)	0.13590638159807358
  (0, 21430)	0.10634484082268654
  (0, 110742)	0.13590638159807358
  (0, 22984)	0.044029886168262956
  (0, 26345)	0.03204912311247104
  (0, 52432)	0.07130725706964733
  (0, 9645)	0.051945598059030484
  (0, 21475)	0.06599324683049447
  (0, 102075)	0.09512593614658092
  (0, 53379)	0.06337607693673737
  (0, 1033)	0.04938636119059658
  (0, 82609)	0.13235980247445467
  (0, 69040)	0.1296088619868655
  (0, 110540)	0.13590638159807358
  (0, 14461)	0.13590638159807358
  (0, 7199)	0.12347694786255234
  (0, 28982)	0.04641512655940694
  (0, 97396)	0.14735504171476618
  (0, 72574)	0.07780976202176307
  (0, 59673)	0.024548357718875483
  (0, 104409)	0.05234606128622293
  (0, 23416)	0.09139055210288637
  (0, 22405)	0.08209143138311972
  (0, 99278)	0.08016122920777802
  :	:
  (67688, 96758)	0.03966436834924497
  (67688, 51538)	0.06465679754295264
  (67688, 81981)	0.07061261556386286
  (67688, 96684)	0.04773529766428956
  (67688, 106790

In [13]:
# CountVectorizer
cvectorizer= CountVectorizer()
X_train_cv_brut = cvectorizer.fit_transform(X_train_brut)
X_test_cv_brut = cvectorizer.transform(X_test_brut)

X_train_cv_brut.shape

(67690, 114002)

In [14]:
print(X_train_cv_brut)

  (0, 27260)	1
  (0, 26680)	16
  (0, 71452)	3
  (0, 40144)	1
  (0, 87092)	1
  (0, 58030)	4
  (0, 9014)	3
  (0, 84577)	4
  (0, 109643)	2
  (0, 68349)	2
  (0, 13138)	2
  (0, 68339)	2
  (0, 14661)	2
  (0, 18957)	2
  (0, 25542)	2
  (0, 97088)	1
  (0, 3983)	1
  (0, 14672)	1
  (0, 106354)	1
  (0, 100323)	1
  (0, 110528)	2
  (0, 23702)	1
  (0, 69028)	1
  (0, 101059)	2
  (0, 21911)	1
  :	:
  (67688, 39251)	1
  (67688, 29523)	1
  (67688, 77652)	1
  (67688, 27108)	1
  (67688, 81299)	2
  (67688, 89066)	1
  (67688, 12382)	1
  (67688, 6324)	1
  (67688, 110305)	1
  (67688, 84089)	1
  (67688, 95317)	1
  (67688, 44919)	1
  (67688, 24141)	1
  (67688, 65130)	1
  (67688, 40812)	1
  (67688, 40813)	1
  (67688, 108727)	1
  (67688, 71606)	1
  (67688, 66581)	3
  (67688, 90575)	1
  (67688, 11219)	1
  (67689, 55533)	1
  (67689, 22748)	1
  (67689, 63127)	1
  (67689, 83492)	1


In [15]:
# AVEC STEMMATISATION
# TfidfVectorizer
X_train_tf = tfvectorizer.fit_transform(X_train)
X_test_tf = tfvectorizer.transform(X_test)

X_train_tf.shape

(67690, 49175)

In [16]:
print(X_train_tf)

  (0, 47748)	0.14973994605044025
  (0, 9247)	0.13823112633873075
  (0, 47745)	0.14973994605044025
  (0, 9921)	0.04855101950213774
  (0, 22689)	0.06609072615559547
  (0, 4151)	0.05621428863942108
  (0, 9266)	0.07681181686007454
  (0, 43998)	0.10741244520187068
  (0, 23124)	0.06982696640000989
  (0, 390)	0.05440516630258837
  (0, 35551)	0.14973994605044025
  (0, 29780)	0.14973994605044025
  (0, 47660)	0.14973994605044025
  (0, 6207)	0.14973994605044025
  (0, 3110)	0.0895025093197969
  (0, 12609)	0.05112368218436073
  (0, 41930)	0.13049173783977122
  (0, 31273)	0.07362984850513785
  (0, 10077)	0.09133929044123125
  (0, 9673)	0.09005028386920058
  (0, 42750)	0.09158738807804842
  (0, 35550)	0.1552473656330569
  (0, 15550)	0.04763954730572349
  (0, 20563)	0.14973994605044025
  (0, 15862)	0.08422975149888695
  :	:
  (67688, 9941)	0.10715404127968797
  (67688, 2741)	0.11957145846920882
  (67688, 35255)	0.08163612133551379
  (67688, 3421)	0.2282263132699476
  (67688, 44619)	0.06696509029059852

In [17]:
# CountVectorizer
X_train_cv = cvectorizer.fit_transform(X_train)
X_test_cv = cvectorizer.transform(X_test)

X_train_cv.shape

(67690, 49175)

In [18]:
print(X_train_cv)

  (0, 11819)	1
  (0, 30839)	3
  (0, 17481)	1
  (0, 37492)	1
  (0, 3892)	2
  (0, 36373)	4
  (0, 47302)	2
  (0, 29492)	4
  (0, 5626)	2
  (0, 6293)	2
  (0, 8158)	2
  (0, 11051)	2
  (0, 41796)	1
  (0, 1673)	1
  (0, 6292)	1
  (0, 45903)	1
  (0, 43221)	1
  (0, 47652)	2
  (0, 10243)	1
  (0, 29774)	1
  (0, 9455)	1
  (0, 13003)	1
  (0, 15916)	1
  (0, 3887)	1
  (0, 26444)	1
  :	:
  (67688, 35223)	1
  (67688, 30790)	1
  (67688, 3588)	1
  (67688, 6444)	1
  (67688, 30055)	1
  (67688, 18720)	1
  (67688, 31365)	1
  (67688, 43657)	1
  (67688, 10576)	1
  (67688, 12851)	1
  (67688, 33396)	1
  (67688, 11738)	1
  (67688, 34982)	2
  (67688, 38332)	1
  (67688, 5285)	1
  (67688, 46909)	1
  (67688, 41018)	1
  (67688, 36188)	1
  (67688, 19510)	1
  (67688, 28111)	1
  (67688, 17777)	2
  (67688, 28753)	2
  (67689, 24030)	1
  (67689, 9833)	1
  (67689, 27224)	1


In [19]:
#Modèle lightgbm avec stemmisa
import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(X_train_tf, y_train)
y_pred_clf = clf.predict(X_test_tf)

In [20]:
from sklearn.metrics import accuracy_score
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred_clf)))

LightGBM Model accuracy score: 0.7846


In [26]:
# Matrice de confusion

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_clf)
# Matrice de confusion
pd.crosstab(y_test, y_pred_clf, rownames=['classes réelles'], colnames=['classes prédites'])

classes prédites,10,40,50,60,1140,1160,1180,1280,1281,1300,1301,1302,1320,1560,1920,1940,2060,2220,2280,2403,2462,2522,2582,2583,2585,2705,2905
classes réelles,,,,,,,,,,,,,,,,,,,,,,,,,,,
10,227,8,0,0,1,16,1,3,7,2,0,2,3,3,0,3,1,4,221,80,0,11,1,0,1,22,0
40,26,277,14,0,12,18,2,13,22,0,0,0,3,1,0,1,0,3,63,21,17,4,0,0,0,6,1
50,2,6,266,1,4,4,0,3,2,2,0,0,2,3,0,0,1,1,5,2,25,1,0,1,1,0,1
60,1,3,11,137,0,0,0,0,2,0,0,0,0,0,0,0,0,0,2,0,15,1,0,0,0,0,1
1140,5,18,1,0,407,14,5,23,4,0,0,0,3,0,1,1,4,0,29,8,3,6,0,0,1,0,0
1160,10,13,0,0,8,630,0,0,5,0,2,0,0,0,0,0,2,0,89,6,0,4,0,1,2,0,0
1180,0,2,3,0,10,9,72,6,9,2,0,0,0,2,0,0,2,0,15,3,3,1,1,1,0,2,0
1280,8,16,3,0,60,8,4,716,25,26,1,14,21,14,3,1,17,3,40,11,0,12,2,7,2,5,0
1281,6,13,0,0,13,16,10,71,211,0,0,10,3,2,0,1,2,0,16,2,7,3,0,5,4,5,0


In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_clf))

              precision    recall  f1-score   support

          10       0.43      0.37      0.40       617
          40       0.67      0.55      0.60       504
          50       0.81      0.80      0.80       333
          60       0.97      0.79      0.87       173
        1140       0.73      0.76      0.75       533
        1160       0.81      0.82      0.81       772
        1180       0.73      0.50      0.60       143
        1280       0.74      0.70      0.72      1019
        1281       0.64      0.53      0.58       400
        1300       0.95      0.90      0.92      1038
        1301       0.97      0.90      0.93       156
        1302       0.90      0.76      0.83       505
        1320       0.82      0.71      0.76       631
        1560       0.84      0.86      0.85      1012
        1920       0.91      0.93      0.92       830
        1940       0.83      0.81      0.82       160
        2060       0.81      0.82      0.82       966
        2220       0.82    